In [1]:
from dms_stan.datasets.trpb import TrpBExponentialGrowth, TrpBSigmoidGrowth

SOURCE_FILE = "~/GitRepos/DMSStan/raw_data/trpb/3-site_merged_replicates/LibI/20230926/LibI_merged_AAs.csv"

Prior predictive check for the TrpB exponential growth model:

In [2]:
EXP_MODEL = TrpBExponentialGrowth.from_data_file(SOURCE_FILE)
EXP_MODEL.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'92b0bd91-ec57-4817-8867-9307a58f8f5d': {'version…

Now a slightly more expressive model: Sigmoid growth parametrized using initial abundances:

In [3]:
SIG_MODEL = TrpBSigmoidGrowth.from_data_file(SOURCE_FILE)
SIG_MODEL.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'ded198b3-fdac-4522-9f43-6804c829c9e5': {'version…

# MAP

Now that we've selected our priors, we're ready to identify the MAP for each.

In [4]:
# EXP_MAP = EXP_MODEL.approximate_map(early_stop=10, device=1, seed=1025)
# EXP_MAP.plot_loss_curve()

We can plot the posterior predictive checks for the MAP:

In [5]:
# EXP_MAP.get_inference_obj(batch_size=50).run_ppc(logy_ppc_samples=True)

And for the full sigmoid model:

In [6]:
SIG_MAP = SIG_MODEL.approximate_map(early_stop=10, device=1, seed=1025)
SIG_MAP.plot_loss_curve()

Epochs:  17%|█▋        | 16551/100000 [04:12<21:14, 65.50it/s, -log pdf/pmf=656960.40] 


:Curve   [iteration]   (-log pdf/pmf)

In [7]:
SIG_MAP.get_inference_obj(batch_size=50).run_ppc(logy_ppc_samples=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'bbede924-9ac9-4383-9d53-0f187cc43836': {'version…

# MCMC

Finally, we will use Stan to sample from the posterior. Sampling is likely to take some time with these models, so we're going to compile an object that will allow us to run sampling outside of the notebook:

In [8]:
EXP_MODEL.mcmc(
    output_dir="./exponential",
    model_name="libI",
    seed=1025,
    delay_run=True,
    iter_warmup=2000,
)
SIG_MODEL.mcmc(
    output_dir="./sigmoid",
    model_name="libI",
    seed=1025,
    delay_run=True,
    iter_warmup=2000,
)

18:36:25 - cmdstanpy - INFO - compiling stan file /home/bwittmann/GitRepos/DMSStan/flip3/trpB/exponential/libI.stan to exe file /home/bwittmann/GitRepos/DMSStan/flip3/trpB/exponential/libI


18:36:43 - cmdstanpy - INFO - compiled model executable: /home/bwittmann/GitRepos/DMSStan/flip3/trpB/exponential/libI
18:36:43 - cmdstanpy - WARNING - Stan compiler has produced 3 warnings:
18:36:43 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=libI.stan --warn-pedantic --O1 --include-paths=/home/bwittmann/GitRepos/DMSStan/dms_stan/model/stan --o=/home/bwittmann/GitRepos/DMSStan/flip3/trpB/exponential/libI.hpp /home/bwittmann/GitRepos/DMSStan/flip3/trpB/exponential/libI.stan
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    is provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.

--- Compiling C++ code ---
/home/bwittmann/micromamba/envs/dms_stan/bin/x86_64-conda-linux-gnu-c++ -fvisibility-inlines-hidden -fmessage-le

Now run analysis on the diagnostics and report:

In [9]:
# samples = SampleResults.from_disk("/home/bwittmann/GitRepos/DMSStan/flip3/trpB/sigmoid/model-20250410192656-20250410192733_arviz.nc", skip_fit=True)

In [10]:
# _ = samples.diagnose()

In [11]:
# samples.run_ppc(logy_ppc_samples=True)

In [12]:
# samples.inference_obj